In [1]:
from head import blip_vqa

In [2]:
model = blip_vqa.BLIP_VQA(vit='base_8')

In [1]:
from backbone.utils import _download, _MODELS
_download(_MODELS['base_8'] ,'/Users/valery/GitHub/VideoQA-NTO/models')
_download('https://huggingface.co/bert-base-multilingual-uncased/resolve/main/pytorch_model.bin' ,'/Users/valery/GitHub/VideoQA-NTO/models')

In [3]:
import torch
checkpoint = torch.load('/Users/valery/GitHub/VideoQA-NTO/models/k600_k710_uniformerv2_b16_8x224.pyth', map_location='cpu') 

ww = {}
for key in checkpoint.keys():
    ww[key.replace('backbone', 'visual_encoder')] = checkpoint[key]

msg = model.load_state_dict(ww,strict=False)

In [4]:
import torch
checkpoint = torch.load('/Users/valery/GitHub/VideoQA-NTO/models/pytorch_model.bin', map_location='cpu') 

ww_encoder = {}
for key in model.state_dict().keys():
    key = key.replace('text_encoder', 'bert')
    if key in checkpoint.keys():
        if 'word_embeddings' in key:
            del checkpoint[key]
        else:
            ww_encoder[key.replace('bert', 'text_encoder')] = checkpoint[key]
# for key in checkpoint.keys():
    # if 'word_embeddings' in key:
    #     # checkpoint[key.replace('bert', 'text_encoder')] = model.state_dict()[key.replace('bert', 'text_encoder')]
    # ww_encoder[key.replace('bert', 'text_encoder')] = checkpoint[key]

msg1 = model.load_state_dict(ww_encoder,strict=False)

ww_decoder = {}
for key in checkpoint.keys():
    ww_decoder[key.replace('bert', 'text_decoder.bert')] = checkpoint[key]

msg2 = model.load_state_dict(ww_decoder,strict=False)

In [5]:
key

'cls.seq_relationship.bias'

In [3]:
import cv2
import numpy as np
# Load video file
cap = cv2.VideoCapture('/Users/valery/GitHub/VideoQA-NTO/eating_spaghetti.mp4')

# Set batch size
batch_size = 8

# Load frames in batches
frames = []
while True:
    # Read a batch of frames
    batch_frames = []
    for i in range(batch_size):
        ret, frame = cap.read()
        if not ret:
            break
        batch_frames.append(frame)
    # Check if we have reached end of video
    if not batch_frames:
        break
    # Add batch to list of frames
    frames.append(batch_frames)

# Convert frames to numpy array
frames = np.array(frames)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (38,) + inhomogeneous part.

[[array([[[  6,  21,  64],
          [  6,  21,  64],
          [  6,  21,  64],
          ...,
          [ 27,  57, 103],
          [ 28,  58, 104],
          [ 30,  60, 106]],
  
         [[  6,  21,  64],
          [  6,  21,  64],
          [  6,  21,  64],
          ...,
          [ 27,  57, 103],
          [ 28,  58, 104],
          [ 30,  60, 106]],
  
         [[  6,  21,  64],
          [  6,  21,  64],
          [  6,  21,  64],
          ...,
          [ 27,  57, 103],
          [ 28,  58, 104],
          [ 30,  60, 106]],
  
         ...,
  
         [[ 42,  49,  42],
          [ 41,  48,  41],
          [ 40,  47,  40],
          ...,
          [ 20,  83,  64],
          [ 20,  83,  64],
          [ 20,  83,  64]],
  
         [[ 42,  49,  42],
          [ 41,  48,  41],
          [ 40,  47,  40],
          ...,
          [ 20,  83,  64],
          [ 20,  83,  64],
          [ 20,  83,  64]],
  
         [[ 43,  50,  43],
          [ 42,  49,  42],
          [ 41,  48,  41

In [1]:
import torch
video = torch.randn(1, 3, 8, 224, 224)
question = 'who was gay?'
answer = 'adam'
n = 1
weights = 0.2



In [7]:
outputs = model(video, question, answer, [n] , weights, train=False)

In [8]:
outputs

['нацыянальны jobs ש jonathanasne rekyhanyasne pistas']